In [4]:
# Skaičiuoja, kiek Zive įrašuose yra rpeaks, nesutampančių tarp tų, kurie rasti
# grynai automatiniu būdu ir tų, kurie yra pataisyti anotuojant įrašus ir saugomi json. 
# Rezultatai įrašomi į Excel (Windows atveju) ir csv failus, naudojau šiuos failus darbui  

import neurokit2 as nk
import pandas as pd
import numpy as np
from pathlib import Path
import os, sys, json
from zive_util_vu import zive_read_df_rpeaks, get_rec_file_name, split_SubjCode, get_recId


def misalignments(rec_dir, SubjCode, print_flag=False):

       # I-a dalis: nuskaitome rpeaks iš json failo
    df_rpeaks = zive_read_df_rpeaks(rec_dir, str(SubjCode))
    atr_sample = df_rpeaks['sampleIndex'].to_numpy()
    n_rpeaks = atr_sample.shape[0]
    if (print_flag):
        print('    json:',atr_sample[:10])

        # II-a dalis: suformuojame rpeaks su Neurokitu
    file_path = Path(rec_dir, str(SubjCode) + '.npy')
    signal_raw = np.load(file_path)
    _, rpeaks = nk.ecg_peaks(signal_raw, sampling_rate=200, correct_artifacts=False)
    rpeaks_from_signal = rpeaks['ECG_R_Peaks']
    if (print_flag):
        print('neurokit:',rpeaks_from_signal[:10])
  
    # Sulyginimas
    ab = np.setdiff1d(atr_sample, rpeaks_from_signal)
    ba = np.setdiff1d(rpeaks_from_signal,atr_sample)
    return(n_rpeaks, np.size(ab)+np.size(ba))


pd.set_option("display.max_rows", 6000)
pd.set_option("display.max_columns",200)
pd.set_option('display.width', 1000)

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'
# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\\DI'   # variantas: Windows, GUNDAS
    # Duomenu_aplankas = 'F:\DI\Data\MIT&ZIVE\VU'   # variantas: Windows, HERKULIS
else:
    Duomenu_aplankas = '/home/kesju/DI'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

# Vietinės talpyklos aplankas ir pūpsnių atributų failas
db_folder = 'DUOM_2022_RUDUO_2'

#  Nuoroda į aplanką su MIT2ZIVE duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder)

# Nuoroda į aplanką su EKG įrašais (.npy) ir anotacijomis (.json)
rec_dir = Path(db_path, 'records_npy')

# Variantas: visi duomenys
# Nuskaitome failą info_create.json ir duomenų rinkinio parametrus
file_path = Path(rec_dir,'info_create_z.json')
with open(file_path) as json_file:
    info_create = json.load(json_file)
SubjCodes =  info_create['SubjCodes'] # pacientų įrašų sąrašas

# Variantas: testinė imtis  
# file_path = Path(rec_dir, 'test_subjcode_lst.csv')
# SubjCodes = list(np.loadtxt(file_path, delimiter=',', dtype="int"))

# Pacientų įrašų sąrašas testavimui
# file_path = 'testinis_sarasas.csv'
# SubjCodes = [10000, 10001, 10002] #Testavimui

df_ns = pd.DataFrame({'userNr': pd.Series(dtype='int'),
                   'recordingNr': pd.Series(dtype='int'),
                   'filename': pd.Series(dtype='str'),
                   'userId': pd.Series(dtype='str'),
                   'recId': pd.Series(dtype='str'),
                   'ns': pd.Series(dtype='int')
                   })

# Failas rezultatų išvedimui į failą
filename = "information_about_rpeaks_tst"

tot, ns_tot = 0, 0
dict_ns_tot = []

for SubjCode in SubjCodes:
    n_rpeaks, ns = misalignments(rec_dir, SubjCode, print_flag=False)
    userNr, recordingNr = split_SubjCode(SubjCode)
    userId, recId, file_name = get_recId(rec_dir, userNr, recordingNr)
    dict_ns = {'userNr':userNr, 'recordingNr':recordingNr, 'filename':str(file_name), 'userId':userId, 'recId':recId,'ns':ns}
    dict_ns_tot.append(dict_ns) 
    # if (ns != 0):
    pr_text = f"\nZive įrašas: {SubjCode} filename: {file_name} rpeaks: {n_rpeaks} nesutampančių rpeaks: {int(ns)} "
    print(pr_text)
    # n_rpeaks, ns = misalignments(rec_dir, SubjCode, print_flag=False)
    if (ns != 0):
        ns_tot +=1
    tot +=1

df_ns = pd.DataFrame(dict_ns_tot)
print()
print(df_ns.head())

if (OS == 'Windows'): 
    # Windows - įrašome Excel lentelę su rezultatais
    # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html
    # filepath = Path(pred_path, pred_folder+'.xlsx')
    
    filepath = filename + '.xlsx'
    with pd.ExcelWriter(filepath) as writer:
        df_ns.to_excel(writer, sheet_name='rpeaks')
    print(f'\nFailų sąrašas ir rezultatai įrašyti į:  {filepath}')

# Taip pat įrašome dataframe csv pavidalu 
filepath = filename + '.csv'
df_ns.to_csv(filepath)
print(f'\nFailų sąrašas ir rezultatai įrašyti į:  {filepath}')    

pr_text = f"\nViso įrašų: {tot}  Įrašų su nesutampančiais rpeaks: {ns_tot} ({100*ns_tot/tot:.1f}%)"   
print(pr_text)



OS in my system :  win32

Zive įrašas: 10000 filename: 1630673.825 rpeaks: 766 nesutampančių rpeaks: 1 

Zive įrašas: 10001 filename: 1626941.468 rpeaks: 861 nesutampančių rpeaks: 0 

Zive įrašas: 10002 filename: 1632923.661 rpeaks: 4850 nesutampančių rpeaks: 1 

   userNr  recordingNr     filename                    userId                     recId  ns
0    1000            0  1630673.825  6034c808d6c2740008035ede  6133101f3d08d40385cdc8ce   1
1    1000            1  1626941.468  6034c808d6c2740008035ede  60f945e187cf66e97e37810e   0
2    1000            2  1632923.661  6034c808d6c2740008035ede  615516b62312e50c3f168e38   1

Failų sąrašas ir rezultatai įrašyti į:  information_about_rpeaks_tst.xlsx

Failų sąrašas ir rezultatai įrašyti į:  information_about_rpeaks_tst.csv

Viso įrašų: 3  Įrašų su nesutampančiais rpeaks: 2 (66.7%)
